## Домашнее задание № 14. API and Web-scrapping.

### Задание 1

- [ ] Будем парсить страницу со свежеми новостям на habr.com/ru/all/.


Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

- [ ] Дополнительная часть (необязательная)


Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [13]:
# Импортируем необходимые для работы библиотеки:

import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re


In [14]:
# Определяем набор ключевых слов для поиска статей
KEYWORDS = ['SQL','Python','Райффайзенбанк','BigData']
base_link = 'https://habr.com/ru/all/'

# Делаем get-запрос к основной странице для поиска и превращаем ее с помощью BS
req = requests.get(base_link)
soup = BeautifulSoup(req.text, 'html.parser')

In [15]:
# получаем список статей
posts = soup.find_all('article', class_='post')


In [16]:
# В статьях находим заголовки, чтобы найти ссылку для перехода в полнотекстовую статью

headers = list(map(lambda x: x.find('h2', class_='post__title'), posts))
headers[0]

<h2 class="post__title">
<a class="post__title_link" href="https://habr.com/ru/post/530670/">Ядерная война, скорее всего, не приведёт к вымиранию человечества</a>
</h2>

In [17]:
link_to_full = list(map(lambda x: x.find('a').get('href'), headers))
link_to_full

['https://habr.com/ru/post/530670/',
 'https://habr.com/ru/company/gitlab/blog/530666/',
 'https://habr.com/ru/post/530668/',
 'https://habr.com/ru/post/530658/',
 'https://habr.com/ru/post/530654/',
 'https://habr.com/ru/company/netologyru/blog/530650/',
 'https://habr.com/ru/post/530638/',
 'https://habr.com/ru/company/selectel/blog/530648/',
 'https://habr.com/ru/company/jugru/blog/530584/',
 'https://habr.com/ru/post/529962/',
 'https://habr.com/ru/company/kaspersky/blog/530644/',
 'https://habr.com/ru/post/530636/',
 'https://habr.com/ru/post/528762/',
 'https://habr.com/ru/company/ruvds/blog/529828/',
 'https://habr.com/ru/post/530614/',
 'https://habr.com/ru/company/otus/blog/530612/',
 'https://habr.com/ru/company/habr_career/blog/530586/',
 'https://habr.com/ru/company/skillfactory/blog/530594/',
 'https://habr.com/ru/post/530418/',
 'https://habr.com/ru/company/habr_career/blog/530598/']

In [18]:
# получаем ссылку на последнюю страницу и формируем набор страниц для парсинга
link_to_last = soup.find('a', title='Последняя страница').get('href')
print(link_to_last)
last_page_num = re.search('(page)(\d+)\/',link_to_last)
print(last_page_num.group(1),last_page_num.group(2))


pages_for_scrap = [base_link,]
for i in range(2, int(last_page_num.group(2))+1):
    pages_for_scrap.append(base_link + last_page_num.group(1) + str(i))
print(pages_for_scrap)

/ru/all/page50/
page 50
['https://habr.com/ru/all/', 'https://habr.com/ru/all/page2', 'https://habr.com/ru/all/page3', 'https://habr.com/ru/all/page4', 'https://habr.com/ru/all/page5', 'https://habr.com/ru/all/page6', 'https://habr.com/ru/all/page7', 'https://habr.com/ru/all/page8', 'https://habr.com/ru/all/page9', 'https://habr.com/ru/all/page10', 'https://habr.com/ru/all/page11', 'https://habr.com/ru/all/page12', 'https://habr.com/ru/all/page13', 'https://habr.com/ru/all/page14', 'https://habr.com/ru/all/page15', 'https://habr.com/ru/all/page16', 'https://habr.com/ru/all/page17', 'https://habr.com/ru/all/page18', 'https://habr.com/ru/all/page19', 'https://habr.com/ru/all/page20', 'https://habr.com/ru/all/page21', 'https://habr.com/ru/all/page22', 'https://habr.com/ru/all/page23', 'https://habr.com/ru/all/page24', 'https://habr.com/ru/all/page25', 'https://habr.com/ru/all/page26', 'https://habr.com/ru/all/page27', 'https://habr.com/ru/all/page28', 'https://habr.com/ru/all/page29', 'ht

In [11]:
# в full-статьях собираем необходимые данные

result_df = pd.DataFrame()
for article in link_to_full:
    subreq = requests.get(article)
    time.sleep(0.3)
    subsoup = BeautifulSoup(subreq.text, 'html.parser')
    # получаем полный текст статьи
    post_full_text = subsoup.find('div', class_='post__body post__body_full').text
    # проверяем на наличие ключевых слов
    if any([keyword.lower() in post_full_text.lower() for keyword in KEYWORDS]):
        # находим дату поста, заголовок и полный текст
        post_date = subsoup.find('span', class_='post__time').get('data-time_published')
        post_title = subsoup.find('span', class_='post__title-text').text
        res_string = {'post_date': post_date, 'post_title': post_title, 'URL': article, 'Full_Text': post_full_text}
        result_df = pd.concat([result_df, pd.DataFrame([res_string])])
result_df
    

,post_date,post_title,URL,Full_Text
0,2020-11-30T16:23Z,ESP32 в окружении VSCode,https://habr.com/ru/post/530638/,\nВ нескольких следующих статьях я хотел бы де...
0,2020-11-30T15:30Z,Программа SmartData 2020,https://habr.com/ru/company/jugru/blog/530584/,"\n\nМы уже рассказывали Хабру, что новая Smart..."
0,2020-11-30T13:07Z,CTF-соревнования 2020 для «белых хакеров». Ста...,https://habr.com/ru/company/otus/blog/530612/,\n\n\n\nВ декабре OTUS при поддержке VolgaCTF ...
0,2020-11-30T12:58Z,Как разработать ансамбль Light Gradient Booste...,https://habr.com/ru/company/skillfactory/blog/...,\nВ преддверии старта нового потока курса «Маш...
0,2020-11-30T11:44Z,Макропроблема микросервисов,https://habr.com/ru/company/mailru/blog/530528/,\n\r\nВсего за 20 лет разработка ПО перешла от...


In [19]:
%%time
KEYWORDS = ['SQL','Python','Райффайзенбанк','BigData']
# финальная сборка по всем страницам
result_df = pd.DataFrame()
for page in pages_for_scrap:
    mainreq = requests.get(page)
    time.sleep(0.3)
    mainsoup = BeautifulSoup(mainreq.text, 'html.parser')
    # получаем список постов
    posts = mainsoup.find_all('article', class_='post')
    # в постах берем заголовки
    headers = list(map(lambda x: x.find('h2', class_='post__title'), posts))
    # в заголовках находим ссылки на полные тексты статей:
    link_to_full = list(map(lambda x: x.find('a').get('href'), headers))
    # по ссылкам страницы переходим и ищем статьи, которые подойдут нам по ключевым словам
    for article in link_to_full:
        subreq = requests.get(article)
        time.sleep(0.3)
        subsoup = BeautifulSoup(subreq.text, 'html.parser')
        # получаем полный текст статьи
        post_full_text = subsoup.find('div', class_='post__body post__body_full').text
        # проверяем на наличие ключевых слов
        if any([keyword.lower() in post_full_text.lower() for keyword in KEYWORDS]):
            # находим дату поста, заголовок и полный текст
            post_date = subsoup.find('span', class_='post__time').get('data-time_published')
            post_title = subsoup.find('span', class_='post__title-text').text
            res_string = {'post_date': post_date, 'post_title': post_title, 'URL': article, 'Full_Text': post_full_text}
            result_df = pd.concat([result_df, pd.DataFrame([res_string])])
            
result_df

Wall time: 20min 32s


,post_date,post_title,URL,Full_Text
0,2020-11-30T19:15Z,Новые ограничения в использовании Docker Hub и...,https://habr.com/ru/company/gitlab/blog/530666/,"\nНи для кого уже не новость, что начиная с 2 ..."
0,2020-11-30T16:23Z,ESP32 в окружении VSCode,https://habr.com/ru/post/530638/,\nВ нескольких следующих статьях я хотел бы де...
0,2020-11-30T15:30Z,Программа SmartData 2020,https://habr.com/ru/company/jugru/blog/530584/,"\n\nМы уже рассказывали Хабру, что новая Smart..."
0,2020-11-30T13:07Z,CTF-соревнования 2020 для «белых хакеров». Ста...,https://habr.com/ru/company/otus/blog/530612/,\n\n\n\nВ декабре OTUS при поддержке VolgaCTF ...
0,2020-11-30T12:58Z,Как разработать ансамбль Light Gradient Booste...,https://habr.com/ru/company/skillfactory/blog/...,\nВ преддверии старта нового потока курса «Маш...
...,...,...,...,...
0,2020-11-13T10:05Z,Релиз Apache Ignite 2.9.0 — что нового?,https://habr.com/ru/company/sberbank/blog/527870/,\nApache Ignite – это высокопроизводительная р...
0,2020-11-13T06:30Z,Обзор операторов PostgreSQL для Kubernetes. Ча...,https://habr.com/ru/company/flant/blog/527524/,"\n\n\r\nНа прошлую статью, где мы рассмотрели ..."
0,2020-11-12T16:00Z,Ультимативный гайд по собеседованию DevOps-инж...,https://habr.com/ru/company/rebrainme/blog/527...,"\n\n\r\nЯ начал заниматься сетями еще в школе,..."
0,2020-11-12T15:53Z,"Hey, Google: умные устройства будут активирова...",https://habr.com/ru/company/selectel/blog/527788/,\n\nPhoto by Cristian Cristian on Unsplash\n\r...


In [22]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 0 to 0
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_date   193 non-null    object
 1   post_title  193 non-null    object
 2   URL         193 non-null    object
 3   Full_Text   193 non-null    object
dtypes: object(4)
memory usage: 7.5+ KB
